In [10]:
import pandas as pd
import re

In [11]:
df = pd.read_csv('../output/test_set_with_answers_lambda_75.csv')

Gdy w kolumnach `unigram_case` i/lub `bigram_case` znajduje się `NaN` oznacza to, że model kanału z błędem zwrócił dla analizowanej operacji `None`. `None` wiążę się z: 
 1. brakiem kandydatów (wygenerowane przez algorytm DL słowa nie znajdują się na liście istniejących słów języka polskiego) 
 2. błędem w zbiorze testowym określono ciąg jedynie zakazanych znaków (tzn. innych niż litery alfabetu łacińskiego rozszerzonego o polskie znaki diakrytyczne) - po przeprowadzeniu czyszczenia powstaje pusty napis, z którego nie można generować sensownych nowych słów
 3. wskazany błąd nie znajduje się w zdaniu
 
Przypadki 2 oraz 3 są rzadkie - stanowią nie więcej niż 30 obserwacji.

In [12]:
df.head()

,Unnamed: 0,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,gold_standard,unigram_case,bigram_case
0,404,Milicja zaczęła przesłuchiwać wszystkich człon...,Milicja zaczęła przesłuchiwać wszystkich człon...,True,wydawnict,nonword,1,pisownia,plewic.08.0100.yaml,delete,"(9, 9)",wydawnictw,wydawnictw,wydawnictw
1,370,Guz olbrzymiokomórkowy kości (łac. tumor gigan...,Guz olbrzymiokomórkowy kości (łac. tumor gigan...,True,chrakteryzującym,nonword,1,pisownia,plewic.05.0014.yaml,delete,"(2, 2)",charakteryzującym,NaN,NaN
2,48,"Spośród możliwych populacji Evans, Lahn i inni...","Spośród możliwych populacji Evans, Lahn i inni...",True,wskzują,nonword,1,pisownia,plewic.06.0019.yaml,delete,"(3, 3)",wskazują,wskazują,wskazują
3,144,Pierwsza randka J.D. z Kailie nie przebiegła p...,Pierwsza randka J.D. z Kailie nie przebiegła p...,True,okazjii,nonword,1,pisownia,plewic.05.0086.yaml,insert,"(5, 6)",okazji,okazji,okazji
4,26,Posłowie na Sejm II Dubois Stanislaw,Posłowie na Sejm II Dubois Stanisław,False,Stanislaw,nonword,1,znaki diakrytyczne,plewic.01.0316.yaml,replace,"(6, 6)",stanisław,NaN,NaN


In [13]:
df_unigram_nan = df.loc[df['unigram_case'].isna()]

In [14]:
df_bigram_nan = df.loc[df['unigram_case'].isna()]

Obserwacje z `NaN` odpowiadają sobie w przypadku metody 1-gramów oraz 2-gramów:

In [15]:
set(df_unigram_nan.index) - set(df_bigram_nan.index)

set()

In [16]:
set(df_bigram_nan.index) - set(df_unigram_nan.index)

set()

Obserwacje, dla których model nie zwrócił odpowiedzi stanowią:

In [17]:
df_unigram_nan.shape[0] / df.shape[0]

0.06797933000712758

Aby poprawić ten wynik można by skorzystać z rozszerzonej listy słów dozwolonych w języku polskim. Jednocześnie na liście muszą znajdować się słowa istniejące, bez błędów, co wymaga kosztownej, manualnej pracy.

In [18]:
df_w_answer = df.dropna(subset=['unigram_case', 'bigram_case'])

In [19]:
df_w_answer.head()

,Unnamed: 0,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,gold_standard,unigram_case,bigram_case
0,404,Milicja zaczęła przesłuchiwać wszystkich człon...,Milicja zaczęła przesłuchiwać wszystkich człon...,True,wydawnict,nonword,1,pisownia,plewic.08.0100.yaml,delete,"(9, 9)",wydawnictw,wydawnictw,wydawnictw
2,48,"Spośród możliwych populacji Evans, Lahn i inni...","Spośród możliwych populacji Evans, Lahn i inni...",True,wskzują,nonword,1,pisownia,plewic.06.0019.yaml,delete,"(3, 3)",wskazują,wskazują,wskazują
3,144,Pierwsza randka J.D. z Kailie nie przebiegła p...,Pierwsza randka J.D. z Kailie nie przebiegła p...,True,okazjii,nonword,1,pisownia,plewic.05.0086.yaml,insert,"(5, 6)",okazji,okazji,okazji
5,264,Generalny remont miał miejsce w 1908 roku z ok...,Generalny remont miał miejsce w 1908 roku z ok...,True,pokrytop,nonword,1,pisownia,plewic.05.0046.yaml,insert,"(6, 7)",pokryto,pokryto,pokryto
6,186,"Górna część Biłego Dunajca (ul. Batorego, ul. ...","Górna część Białego Dunajca (ul. Batorego, ul....",True,Biłego,nonword,1,pisownia,plewic.03.0060.yaml,delete,"(2, 2)",białego,byłego,byłego


### Poziom ogólny - cała ramka danych

Sprawdzam dla różnych wartości współczynnika lambda to czy metoda 1-gramowa oraz 2-gramowa zwracają różne wyniki:

In [21]:
def check_unigram_bigram_answer(csv_file):
    df = pd.read_csv('./Channel_model/{}'.format(csv_file))
    
    # ignore NaN
    df_w_answer = df.dropna(subset=['unigram_case', 'bigram_case'])
    
    return df_w_answer.loc[df_w_answer['unigram_case'] != df_w_answer['bigram_case']]

Dla żadnego poziomu lambda metoda 1-gramowa nie zwraca innej odpowiedzi niż 2-gramowa, dlatego dalej będę analizował tylko kolumnę `unigram_case`. Być może po przeanalizowaniu potencjalnego problemu będę musiał przeprowadzić oddzielną analizę dla `bigram_case`.

In [22]:
check_unigram_bigram_answer('test_set_with_answers_lambda_75.csv')

FileNotFoundError: [Errno 2] File b'./Channel_model/test_set_with_answers_lambda_75.csv' does not exist: b'./Channel_model/test_set_with_answers_lambda_75.csv'

Sprawdzam poprawność na zasadzie równości z `gold_standard`:

In [23]:
df_correct = df_w_answer.loc[df_w_answer['gold_standard'] == df_w_answer['unigram_case']]

Metoda osiąga skuteczność:

In [24]:
df_correct.shape[0] / df_w_answer.shape[0]

0.7863618258583606

In [25]:
df_incorrect = df_w_answer.loc[df_w_answer['gold_standard'] != df_w_answer['unigram_case']]

Wprowadzam kolumnę, która informuje o poprawności odpowiedzi:

In [26]:
df_w_answer['is_correct'] = df_w_answer.apply(lambda x: int(x['gold_standard'] == x['unigram_case']), axis=1)

/home/pawelhdd/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
grouped_df = df_w_answer.groupby(['type', 'category', 'basic_type_operation']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df

is_correct
                                                                mean
type     category                    basic_type_operation           
nonword  znaki diakrytyczne          insert                 1.000000
realword znaki diakrytyczne/kontekst insert                 1.000000
nonword  znaki diakrytyczne          replace                0.974850
realword znaki diakrytyczne/kontekst replace                0.895007
nonword  znaki diakrytyczne          delete                 0.888889
         pisownia                    replace                0.770957
                                     transpose              0.759442
                                     insert                 0.750700
                                     delete                 0.618685
realword fleksja/liczba              replace                0.455042
         znaki diakrytyczne/kontekst delete                 0.441176
         fleksja/liczba              delete                 0.276627
         składnia                    replace                0.205207
                                     insert                 0.201245
                                     delete                 0.042857
         fleksja/liczba              insert                 0.011765
                                     transpose              0.000000
         składnia                    transpose              0.000000

In [32]:
grouped_df_type = df_w_answer.groupby(['type']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_type

,is_correct
,mean
type,
nonword,0.807578
realword,0.640086


In [34]:
grouped_df_category = df_w_answer.groupby(['category']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_category

,is_correct
,mean
category,
znaki diakrytyczne,0.974762
znaki diakrytyczne/kontekst,0.890360
pisownia,0.699725
fleksja/liczba,0.357523
składnia,0.110360


In [35]:
grouped_df_operation = df_w_answer.groupby(['basic_type_operation']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_operation

,is_correct
,mean
basic_type_operation,
replace,0.893954
transpose,0.752585
insert,0.710831
delete,0.583681


Sprawdzam liczebności poszczególnych grup definiowanych przez różne wartości zmiennych kategorycznych:

In [53]:
grouped_df_type_category_count = df_w_answer.groupby(['type','category']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
grouped_df_type_category_count

count
type     category                          
nonword  pisownia                     66639
         znaki diakrytyczne           42990
realword znaki diakrytyczne/kontekst   9668
         fleksja/liczba                3569
         składnia                      2664

In [54]:
grouped_df_type_basic_type_op_count = df_w_answer.groupby(['type', 'basic_type_operation']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
grouped_df_type_basic_type_op_count

count
type     basic_type_operation       
nonword  replace               59119
         delete                28709
         insert                14996
realword replace               12586
nonword  transpose              6805
realword delete                 2248
         insert                 1005
         transpose                62

In [28]:
for i in range(grouped_df.shape[0]):
    print(re.sub("/| ", "_", " ".join(grouped_df.iloc[i].name)), grouped_df.iloc[i].values[0])

nonword_znaki_diakrytyczne_insert 1.0
realword_znaki_diakrytyczne_kontekst_insert 1.0
nonword_znaki_diakrytyczne_replace 0.9748503830659246
realword_znaki_diakrytyczne_kontekst_replace 0.8950068041452947
nonword_znaki_diakrytyczne_delete 0.8888888888888888
nonword_pisownia_replace 0.7709569732937686
nonword_pisownia_transpose 0.7594415870683321
nonword_pisownia_insert 0.7507002801120448
nonword_pisownia_delete 0.6186854591124756
realword_fleksja_liczba_replace 0.4550420168067227
realword_znaki_diakrytyczne_kontekst_delete 0.4411764705882353
realword_fleksja_liczba_delete 0.27662721893491127
realword_składnia_replace 0.20520673813169985
realword_składnia_insert 0.2012448132780083
realword_składnia_delete 0.04285714285714286
realword_fleksja_liczba_insert 0.011764705882352941
realword_fleksja_liczba_transpose 0.0
realword_składnia_transpose 0.0


In [29]:
def create_filtered_dfs(grouped_df=grouped_df, df_to_filter=df_incorrect):
    """Return a dictionary of dataframes."""
    dfs_dict = dict()
    
    for i in range(grouped_df.shape[0]):
    
        # key indicates type, category and basic_type_operation
        col_tuple = grouped_df.iloc[i].name
        key = re.sub("/| ", "_", " ".join(col_tuple))

        # value is a dataframe filtered with key elements
        mask = (df_to_filter['type'] == col_tuple[0]) & (df_to_filter['category'] == col_tuple[1]) & (df_to_filter['basic_type_operation'] == col_tuple[2])

        # get percentage of correct answers for each category
        percentage = grouped_df.iloc[i].values[0]

        dfs_dict[key] = (df_to_filter[mask], percentage)
        
    return dfs_dict

In [30]:
dataframes_filtered_dict = create_filtered_dfs()

In [48]:
dataframes_filtered_dict['realword_składnia_transpose'][0].head(10)

,Unnamed: 0,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,gold_standard,unigram_case,bigram_case
119,349,W biegu na 15 km jego starta do Thomasa Wassbe...,W biegu na 15 km jego strata do Thomasa Wassbe...,True,starta,realword,1,składnia,plewic.08.0044.yaml,transpose,"(3, 2)",strata,startą,startą
840,79,"Jest wieloletnim graczem Athleticu Bilbao, dal...","Jest wieloletnim graczem Athleticu Bilbao, dla...",True,dal,realword,1,składnia,plewic.09.0124.yaml,transpose,"(2, 1)",dla,dał,dał
1173,94,Współpracował z burmistrzem Janem Rajchlem w k...,Współpracował z burmistrzem Janem Rajchlem w k...,True,palny,realword,1,składnia,plewic.09.0042.yaml,transpose,"(2, 1)",plany,palnę,palnę
3466,345,"Współpracując z nadajnikami radiowymi, zapewni...","Współpracując z nadajnikami radiowymi, zapewni...",True,kliku,realword,1,składnia,plewic.04.0037.yaml,transpose,"(2, 1)",kilku,gliku,gliku
3906,288,Dal większej dokładności pomiaru skonstruowano...,Dla większej dokładności pomiaru skonstruowano...,True,Dal,realword,1,składnia,plewic.09.0045.yaml,transpose,"(2, 1)",dla,dał,dał
5779,11,"! 'Rozdaje pomnażania: schizotomia, schizogoni...","! 'Rodzaje pomnażania: schizotomia, schizogoni...",True,Rozdaje,realword,1,składnia,plewic.04.0039.yaml,transpose,"(3, 2)",rodzaje,rozdaję,rozdaję
7768,248,Jest jednym z kliku smartphonów pozwalających ...,Jest jednym z kilku smartphonów pozwalających ...,True,kliku,realword,1,składnia,plewic.04.0088.yaml,transpose,"(2, 1)",kilku,gliku,gliku
12413,318,Wówczas na lewicy doszło do znacznych kłótni p...,Wówczas na lewicy doszło do znacznych kłótni p...,True,strać,realword,1,składnia,plewic.09.0117.yaml,transpose,"(3, 2)",starć,strac,strac
12862,239,miejscowość i wyjechał do U.S.A. Ale on tam je...,miejscowość i wyjechał do U.S.A. Ale on tam je...,False,tka,realword,1,składnia,plewic.03.0043.yaml,transpose,"(2, 1)",tak,ćka,ćka
13031,218,Imigranci ci byli z reguły zamieszani w konfli...,Imigranci ci byli z reguły zamieszani w konfli...,True,tel,realword,1,składnia,plewic.05.0063.yaml,transpose,"(2, 1)",tle,teł,teł
